In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io

In [2]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [3]:
import preprocessing as prep
target_size = (150, 150)
X, y = prep.get_X_y(path=path, percent=0.5, target_size=target_size)

In [4]:
y[:5]

,battery,biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X.shape

(3720, 150, 150, 3)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 2790 2790
Test Lengths: 930 930


In [ ]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size + (3,) # adding 
# removing images from the dataset
# dropout: randomly remove nodes in each layer to create gaps
# normalization layer
# augmentation
# convert to grayscale <- 
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(12, activation='sigmoid')
])

c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 337ms/step - accuracy: 0.1572 - loss: 2.7168 - val_accuracy: 0.3989 - val_loss: 1.7927
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 28s 313ms/step - accuracy: 0.4246 - loss: 1.6696 - val_accuracy: 0.4452 - val_loss: 1.6632
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 29s 333ms/step - accuracy: 0.5751 - loss: 1.3057 - val_accuracy: 0.5247 - val_loss: 1.4708
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 28s 320ms/step - accuracy: 0.6454 - loss: 1.0673 - val_accuracy: 0.4731 - val_loss: 1.6316
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 359ms/step - accuracy: 0.7341 - loss: 0.8033 - val_accuracy: 0.4903 - val_loss: 1.7980
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 365ms/step - accuracy: 0.7933 - loss: 0.6022 - val_accuracy: 0.5269 - val_loss: 1.9256
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 37s 313ms/step - accuracy: 0.8354 - loss: 0.5269 - val_accuracy: 0.5355 - val_loss: 2.0583
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 32s 362ms/step - accuracy: 0.8743 - loss: 0.3690 - val_accu

In [12]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.5205 - loss: 2.9359


[2.987243890762329, 0.5107526779174805]

In [14]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)

### Testing the model

In [4]:
# reloaing the model
import pickle
with open("initial_model.pkl", "rb") as f:
    model = pickle.load(f)

In [5]:
# preprocess a new image 
import preprocessing as prep
image_path ="clothes_test_image.jpg"
X_input = prep.get_X([image_path], target_size=(150, 150))

In [6]:
import pandas as pd
output = model.predict(X_input)
categories = [
    "battery", "biological", "brown-glass", "cardboard", "clothes",
    "green-glass", "metal", "paper", "plastic", "shoes", "trash", "white-glass"
]
pd.DataFrame(output, columns=categories)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 150, 150, 3))', ())
  warnings.warn(msg)


,battery,biological,brown-glass,cardboard,clothes,green-glass,metal,paper,plastic,shoes,trash,white-glass
0,0.994132,0.000021,0.000384,0.000003,1.0,0.000116,0.998827,0.834961,0.953145,0.989013,0.241109,0.959593


In [7]:
import numpy as np
predicted_class = np.argmax(output[0])
categories[predicted_class]

'clothes'

In [8]:
prep.get_prediction(image_path=image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 150, 150, 3))', ())
  warnings.warn(msg)


'clothes'